In [53]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from string import punctuation
import gensim
import pymorphy2

punctuation += '-–—«»'

## Задание 1

In [3]:
morph = pymorphy2.MorphAnalyzer()

In [4]:
text = "Зонды будут запущены в космос в 2029 году и отправятся ко второй точке Лагранжа в системе Солнце"

In [5]:
tokens = text.split()

In [11]:
pos_tags = {token: morph.parse(token)[0].tag.POS for token in tokens}

In [13]:
pos_tags_verbs = {token: pos_tag for token, pos_tag in pos_tags.items() if pos_tag == 'VERB'}

In [17]:
print(', '.join(pos_tags_verbs.keys()))

будут, отправятся


## Задание 2

In [18]:
import requests
r = requests.get(
    'https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/anna_karenina.txt'
)

# работайте с этими предложениями
sentences = r.text.split('\n')
sentences = [sent for sent in sentences if len(sent) > 10]

In [19]:
sentences[0]

'«Анна Каренина» – это сложное, психологически утонченное, остропроблемное произведение, насыщенное приметами времени. Л.Н. Толстой на страницах произведения показывает, как рушатся остатки патриархального уклада жизни в России под натиском буржуазного прогресса, как падают нравы, ослабевают семейные устои, вырождается аристократия.'

In [21]:
vect = TfidfVectorizer()

sentences = [sent.lower() for sent in sentences if sent]

vect_sentences = vect.fit_transform(sentences)

In [24]:
vect.get_feature_names()

['11',
 '17015',
 '18',
 '18308',
 '1863',
 '1864',
 '28',
 '30',
 '36',
 '6лизкий',
 '99',
 'aber',
 'ablutions',
 'admirable',
 'adore',
 'age',
 'ai',
 'aise',
 'alexandre',
 'aline',
 'all',
 'allemand',
 'aller',
 'allons',
 'alphonse',
 'ami',
 'amis',
 'amour',
 'an',
 'and',
 'anglais',
 'anglaise',
 'anne',
 'appetit',
 'arbeitscur',
 'arrivee',
 'assez',
 'assiette',
 'atelier',
 'au',
 'auch',
 'ausgerechnet',
 'ausrechnen',
 'baby',
 'bateau',
 'battre',
 'bear',
 'beau',
 'beaufrere',
 'begier',
 'belle',
 'berthe',
 'bezwungen',
 'bien',
 'bijou',
 'blague',
 'blanche',
 'bonheur',
 'bonhomie',
 'bonne',
 'bottes',
 'bras',
 'breakfast',
 'brin',
 'brise',
 'broderie',
 'broyer',
 'ca',
 'cafes',
 'cancan',
 'canut',
 'carte',
 'ce',
 'cela',
 'celle',
 'ces',
 'cessez',
 'chaine',
 'chambres',
 'champagne',
 'chance',
 'chantants',
 'chaperon',
 'charmante',
 'charmants',
 'chat',
 'chateau',
 'cher',
 'chere',
 'choses',
 'clair',
 'cochon',
 'coeur',
 'coiffer',
 'coif

In [30]:
vect_df = pd.DataFrame(vect_sentences.toarray(), columns=vect.get_feature_names())

In [43]:
vect_df.iloc[:, vect_df.iloc[0].argmax()].name

'буржуазного'

## Задание 3

In [101]:
from razdel import tokenize
import gensim
from collections import Counter

In [102]:
tokenized_sentences = [[token.text for token in tokenize(sent) if token.text not in punctuation] for sent in sentences]

In [103]:
tokenized_sentences[0][:10]

['анна',
 'каренина',
 'это',
 'сложное',
 'психологически',
 'утонченное',
 'остропроблемное',
 'произведение',
 'насыщенное',
 'приметами']

In [104]:
vocab = Counter()
token_count = 0

for sent in tokenized_sentences:
    token_count += len(sent)
    vocab.update(sent)

In [105]:
probas_vocab_unigram = Counter({word:c/token_count for word, c in vocab.items()})

In [106]:
probas_vocab_unigram

Counter({'анна': 0.0018449161284120781,
         'каренина': 0.0001700724286712537,
         'это': 0.008218934976873846,
         'сложное': 1.1091680130733936e-05,
         'психологически': 3.6972267102446456e-06,
         'утонченное': 7.394453420489291e-06,
         'остропроблемное': 3.6972267102446456e-06,
         'произведение': 1.1091680130733936e-05,
         'насыщенное': 3.6972267102446456e-06,
         'приметами': 3.6972267102446456e-06,
         'времени': 0.00025880586971712517,
         'л': 7.394453420489291e-06,
         'н': 2.5880586971712517e-05,
         'толстой': 1.8486133551223227e-05,
         'на': 0.013295227250039746,
         'страницах': 3.6972267102446456e-06,
         'произведения': 1.1091680130733936e-05,
         'показывает': 2.2183360261467872e-05,
         'как': 0.009838320275961002,
         'рушатся': 3.6972267102446456e-06,
         'остатки': 1.1091680130733936e-05,
         'патриархального': 3.6972267102446456e-06,
         'уклада': 3.69

In [107]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [110]:
bigram_vocab = Counter()
bigram_count = 0

for sent in tokenized_sentences:
    bigram_count += len(ngrammer(sent))
    bigram_vocab.update(ngrammer(sent))

In [112]:
probas_vocab_bigram = Counter({word:c/bigram_count for word, c in bigram_vocab.items()})

In [129]:
target_word = 'красный'

In [130]:
target_word_prob = probas_vocab_unigram[target_word]

In [131]:
target_word_bigram_probs = {bigram: prob for bigram, prob in probas_vocab_bigram.items() if bigram.startswith(target_word)}

In [136]:
{bigram: np.log(prob / target_word_prob) for bigram, prob in target_word_bigram_probs.items()}

{'красный мешочек': -0.6648893621687376,
 'красный огонь': -2.274327274602838,
 'красный как': -2.274327274602838,
 'красный платок': -2.274327274602838,
 'красный распотевший': -2.274327274602838}

## Задание 4

In [137]:
words = ["решение","ршеение","ренешик","рещиние","ришение"]

In [138]:
import textdistance

In [139]:
dl_dists_list = []
l_dists_list = []

for word_1 in words:
    for word_2 in words:
        dl_dist = textdistance.damerau_levenshtein(word_1, word_2)
        l_dist = textdistance.levenshtein(word_1, word_2)
        if dl_dist != l_dist:
            print(word_1, word_2, sep=', ')
        dl_dists_list.append(dl_dist)
        l_dists_list.append(l_dist)

решение, ршеение
ршеение, решение


## Задание 5

In [57]:
from transformers import BertTokenizer, BertModel

model = BertModel.from_pretrained('distilbert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased')

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertModel: ['distilbert.transformer.layer.3.attention.q_lin.weight', 'distilbert.transformer.layer.2.attention.out_lin.weight', 'distilbert.transformer.layer.5.ffn.lin1.bias', 'distilbert.transformer.layer.2.ffn.lin1.weight', 'distilbert.transformer.layer.2.sa_layer_norm.weight', 'distilbert.transformer.layer.4.attention.k_lin.bias', 'distilbert.transformer.layer.4.attention.k_lin.weight', 'distilbert.transformer.layer.1.attention.out_lin.bias', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.embeddings.word_embeddings.weight', 'distilbert.transformer.layer.4.sa_layer_norm.bias', 'distilbert.transformer.layer.5.output_layer_norm.bias', 'distilbert.transformer.layer.4.attention.q_lin.bias', 'distilbert.transformer.layer.4

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [60]:
tokenized_corpus = [tokenizer.tokenize(sent) for sent in sentences]

In [61]:
tokenized_corpus[0]

['«',
 'а',
 '##н',
 '##на',
 'к',
 '##а',
 '##р',
 '##е',
 '##н',
 '##и',
 '##на',
 '»',
 '–',
 'э',
 '##т',
 '##о',
 'с',
 '##л',
 '##о',
 '##ж',
 '##н',
 '##о',
 '##е',
 ',',
 'п',
 '##с',
 '##и',
 '##х',
 '##о',
 '##л',
 '##о',
 '##г',
 '##и',
 '##ч',
 '##е',
 '##с',
 '##к',
 '##и',
 'у',
 '##т',
 '##о',
 '##н',
 '##ч',
 '##е',
 '##н',
 '##н',
 '##о',
 '##е',
 ',',
 'о',
 '##с',
 '##т',
 '##р',
 '##о',
 '##п',
 '##р',
 '##о',
 '##б',
 '##л',
 '##е',
 '##м',
 '##н',
 '##о',
 '##е',
 'п',
 '##р',
 '##о',
 '##и',
 '##з',
 '##в',
 '##е',
 '##д',
 '##е',
 '##н',
 '##и',
 '##е',
 ',',
 'н',
 '##а',
 '##с',
 '##ы',
 '##щ',
 '##е',
 '##н',
 '##н',
 '##о',
 '##е',
 'п',
 '##р',
 '##и',
 '##м',
 '##е',
 '##т',
 '##а',
 '##м',
 '##и',
 'в',
 '##р',
 '##е',
 '##м',
 '##е',
 '##н',
 '##и',
 '.',
 'л',
 '.',
 'н',
 '.',
 'т',
 '##о',
 '##л',
 '##с',
 '##т',
 '##о',
 '##и',
 'н',
 '##а',
 'с',
 '##т',
 '##р',
 '##ан',
 '##и',
 '##ц',
 '##а',
 '##х',
 'п',
 '##р',
 '##о',
 '##и',
 '##з',
 '##в',
 

In [62]:
ft_model = gensim.models.FastText(tokenized_corpus, window=5, vector_size=300, min_n=2, max_n=7, sg=0)

In [66]:
ft_model.wv.most_similar('каренина')

[('##ка', 0.5764524340629578),
 ('а', 0.528275191783905),
 ('landau', 0.4276014268398285),
 ('##ович', 0.3830831050872803),
 ('–', 0.3238479197025299),
 ('##на', 0.3101428151130676),
 ('mon', 0.30935391783714294),
 ('au', 0.30712148547172546),
 ('tan', 0.30391260981559753),
 ('pardon', 0.2982883155345917)]